<a href="https://colab.research.google.com/github/ribesstefano/ml4justice-feature-predictor/blob/main/notebooks/ml4justice_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML for Justice

## Setup and Imports

In [1]:
!pip install -U datasets evaluate -qqq
!pip install -U accelerate -qqq
!pip install -U transformers -qqq
!pip install -U sentence-transformers -qqq
!pip install umap-learn -qqq
!pip install plotly==5.18.0 -qqq
!pip install -U optuna -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 52.3 MB/s eta 0:00:00
ERROR: pip's dependency resolve

In [2]:
!pip install bitsandbytes -qqq
!pip install peft -qqq
# !pip install git+https://github.com/huggingface/peft.git -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 4.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

dataset_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/processed/dataset-sentenze-omicidio-stradale.csv", index_col=0, encoding='utf-8')
dataset_df

,corte,numero sentenza,localita commerciale o luogo di lavoro,vittima eta avanzata,vittima eta giovane,lavori stradali di manutenzione,guida senza patente valida imputato,guida senza patente valida vittima,piu imputati,strada a doppio senso di marcia,...,andamento retromarcia imputato,fuga imputato,uso telefono alla guida,vittima perde controllo,imputato perde controllo,condanna,filename,text,raw text,facts
casi,,,,,,,,,,,,,,,,,,,,,
1,Cass. Penale,20253,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,20253 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 03/05/2...,"1. La Corte d'Appello di Ancona, con sentenza ..."
2,Cass. penale,18284,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,18284 cass. pen.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 20/04/2...,"1. Con la sentenza indicata in epigrafe, la Co..."
3,Cass. penale,15832,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,15832 cass penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 05/04/2...,"1. La Corte d'appello di Messina, in parziale ..."
4,Cass. penale,17209,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,17209 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 04/04/2...,"1. La Corte d'appello di Lecce, Sezione distac..."
5,Corte Appello Ancona,2112,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,2112 corte appello Ancona.pdf,"--- Incipit ::: Corte appello - Ancona, 23/01...","SENTENZA \nCorte appello - Ancona, 23/01/2023,...","Incipit Corte appello - Ancona, 23/01/2023, n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,cass. Penale,19944,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,Cass. penale 19944.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 28/02/2...,1. L.G. ricorre avverso la sentenza della Cort...
78,cass. Penale,42017,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,Cass. penale 42017.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 29/09/2...,1. La Corte d'appello di Bologna con la pronun...
79,cass. Penale,44364,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,Cass. penale 44364.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 26/10/2...,1. Con sentenza del 23 novembre 2022 la Corte ...


## Text Utilities

In [5]:
sentence_div = '|||'
section_div = '---'
section_name_div = ':::'

def clean_text(text):
    text = text.replace(sentence_div, "").replace(section_div, "")
    return text.replace(section_name_div, "").strip()

def clean_section_text(text):
    return text.replace(sentence_div, "").strip()

def get_senteces_from_text(text, clean=True):
    sentences = text.split(sentence_div)
    sentences = [s.split(section_name_div)[-1].strip() for s in sentences]
    if clean:
        sentences = [clean_section_text(s) for s in sentences]
    return sentences

def get_sections_from_text(text):
    sections = text.split(section_div)
    return {s.split(section_name_div)[0].strip(): s.split(section_name_div)[-1].strip() for s in sections}

for i, sample in dataset_df.iterrows():
    text = sample['text']
    # sentences = get_senteces_from_text(text)
    # for s in sentences:
    #     print(s)
    sections = get_sections_from_text(text)
    for k, v in sections.items():
        print(k, clean_section_text(v))
    break

 
Incipit Cassazione penale sez. IV - 03/05/2023, n. 20253 Intestazione LA CORTE SUPREMA DI CASSAZIONE  SEZIONE QUARTA PENALE  Composta dagli Ill.mi Sigg.ri Magistrati: Dott. PICCIALLI Patrizia - Presidente - Dott. PEZZELLA Vincenzo - Consigliere - Dott. D'ANDREA Alessadro - Consigliere - Dott. MICCICHE'Loredana - rel. Consigliere - Dott. NOCERA Andrea - Consigliere - ha pronunciato la seguente:
SENTENZA sul ricorso proposto da:  H.M. nato il (Omissis); avverso la sentenza del 12/04/2022 della CORTE APPELLO di ANCONA visti gli atti, il provvedimento impugnato e il ricorso; udita la relazione svolta dal Consigliere LOREDANA MICCICHE'; lette le conclusioni Pubblico Ministero, in persona del Sostituto Procuratore ASSUNTA COCOMELLO; lette le conclusioni dell'avv. Emilio Perfetti per la parte civile Associazione Italiana familiari e vittime della strada.
RITENUTO IN FATTO 1. La Corte d'Appello di Ancona, con sentenza in data 12 aprile 2022, confermava la sentenza del GUP presso il Tribunale

In [6]:
import random

random.seed(42)

def chunk_section(text, chunk_size=1024):
    """ Group the sections sentence into chunks of size chunk_size """
    # Split the text into sentences
    sentences = get_senteces_from_text(text)
    # Group the sentences into chunks of size chunk_size
    chunks = []
    curr_chunk = ""
    for sentence in sentences:
        if len(curr_chunk) + len(sentence) < chunk_size:
            curr_chunk += sentence + " "
        elif len(sentence) > chunk_size:
            if curr_chunk != "":
                # print(len(curr_chunk))
                chunks.append(curr_chunk)
            # Keep appending chunks of size chunk_size to the list, if the sentence is longer multiple times than chunk_size
            for i in range(0, len(sentence), chunk_size):
                # print(len(sentence[i:i+chunk_size]))
                chunks.append(sentence[i:i+chunk_size])
            curr_chunk = ""
        else:
            # print(len(curr_chunk))
            chunks.append(curr_chunk)
            curr_chunk = sentence + " "
    # print(len(curr_chunk))
    chunks.append(curr_chunk)
    return chunks

def get_facts(text):
    facts = get_sections_from_text(text)
    if 'Svolgimento del processo' in facts and 'Motivi della decisione' in facts:
        # text = facts['Svolgimento del processo'] + " " + facts['Motivi della decisione']
        text = facts['Svolgimento del processo']
    elif "RITENUTO IN FATTO" in facts:
        text = facts['RITENUTO IN FATTO']
    return text

def get_random_chunk(text, chunk_size=1024):
    """ Get a random chunk of size chunk_size from the text """
    text = clean_section_text(text)
    start = random.randint(0, max(0, len(text) - chunk_size - 1))
    # max_chunks = len(text) // chunk_size
    # start = random.randint(0, max_chunks - 1) * chunk_size
    end = start + min(chunk_size, len(text))
    return text[start:end]

for i, sample in dataset_df.iterrows():
    text = sample['text']
    chunk = get_random_chunk(get_facts(text))
    print(chunk)
    break

ione dell'art. 141 C.d.S. poiché alla guida della sua autovettura, nel percorrere la strada provinciale n. (Omissis), alla guida dell'autocarro (Omissis), aveva tenuto una velocità non adeguata e comunque pari a 80 Km/h non commisurata allo stato dei luoghi, in considerazione delle condizioni metereologiche e dell'ora serale, e, affrontando un tratto di strada curvilineo, aveva perso il controllo del veicolo che, capovolgendosi, rovinava contro un manufatto, così cagionando la morte per gravissimo trauma cranico del F.C., seduto al lato passeggero anteriore.  La Corte territoriale, nel respingere i motivi di gravame, riteneva esaustiva la motivazione della sentenza di prime cure in ordine alle modalità del sinistro, su cui si soffermava rispondendo alle osservazioni mosse dall'appellante; confermava la statuizione del primo giudice in ordine al mancato riconoscimento delle circostanze attenuanti generiche, la durata della sanzione amministrativa accessoria e l'ammontare delle spese liq

## Training

### Setup Train/Val/Test Datasets

In [7]:
label_columns = [col for col in dataset_df.columns if col not in ['numero sentenza', 'filename', 'text', 'raw text', 'corte', 'target', 'casi', 'condanna', 'colpa', 'labels', 'score', 'summary', 'facts',]]
label_columns.sort()
for l in label_columns:
    print(l)

andamento retromarcia imputato
attraversamento corretto della vittima
attraversamento vittima con semaforo rosso
attraversamento vittima senza strisce o sbucata improvvisamente
centro abitato
colpo di sonno
eccesso verso imputato
eccesso verso vittima
fuga imputato
giorno
giovane eta imputato
guida brusca o manovra brusca della vittima
guida brusca o manovra brusca imputato
guida senza patente valida imputato
guida senza patente valida vittima
imputato perde controllo
imputato ubriaco o drogato
invasione corsia opposta imputato
lavori stradali di manutenzione
localita commerciale o luogo di lavoro
maltempo
mancanza cintura di sicurezza o casco vittima
neopat imputato
notte
ostacolo evitabile dall imputato
ostacolo prevedibile dall imputato
ostacolo prevedibile dalla vittima
piu imputati
poca illuminazione o scarsa visibilita
presenza rimorchio imputato
prossimita di un incrocio o curva
segnaletica non visibile o assente
strada a doppio senso di marcia
strada rovinata o bagnata
uso tele

Get the test set and the train+val set:

In [8]:
test_size = 10

test_df = dataset_df.drop_duplicates(subset='numero sentenza').sample(n=test_size, random_state=42)
test_idx = test_df['numero sentenza'].tolist()

train_val_df = dataset_df[~dataset_df['numero sentenza'].isin(test_idx)]

display(train_val_df.head(5))
display(test_df.head(5))
print(len(test_df))
print(len(train_val_df))

,corte,numero sentenza,localita commerciale o luogo di lavoro,vittima eta avanzata,vittima eta giovane,lavori stradali di manutenzione,guida senza patente valida imputato,guida senza patente valida vittima,piu imputati,strada a doppio senso di marcia,...,andamento retromarcia imputato,fuga imputato,uso telefono alla guida,vittima perde controllo,imputato perde controllo,condanna,filename,text,raw text,facts
casi,,,,,,,,,,,,,,,,,,,,,
2,Cass. penale,18284,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,18284 cass. pen.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 20/04/2...,"1. Con la sentenza indicata in epigrafe, la Co..."
3,Cass. penale,15832,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,15832 cass penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 05/04/2...,"1. La Corte d'appello di Messina, in parziale ..."
4,Cass. penale,17209,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,17209 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 04/04/2...,"1. La Corte d'appello di Lecce, Sezione distac..."
6,Corte Appello di Napoli,506,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,506 corte d_appello napoli.pdf,--- Incipit ::: Corte appello sez. II - Napol...,"SENTENZA \nCorte appello sez. II - Napoli, 18/...","Incipit Corte appello sez. II - Napoli, 18/01..."
7,Tribunale di Gorizia,395,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,395 tribunale gorizia.pdf,"--- Incipit ::: Tribunale - Gorizia, 15/09/20...","SENTENZA \nTribunale - Gorizia, 15/09/2022, n....","Incipit Tribunale - Gorizia, 15/09/2022, n. 3..."


,corte,numero sentenza,localita commerciale o luogo di lavoro,vittima eta avanzata,vittima eta giovane,lavori stradali di manutenzione,guida senza patente valida imputato,guida senza patente valida vittima,piu imputati,strada a doppio senso di marcia,...,andamento retromarcia imputato,fuga imputato,uso telefono alla guida,vittima perde controllo,imputato perde controllo,condanna,filename,text,raw text,facts
casi,,,,,,,,,,,,,,,,,,,,,
31,Cass. penale,24175,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,24175 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 23/05/2...,"1. La Corte d'Appello di Roma, con sentenza in..."
1,Cass. Penale,20253,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,20253 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 03/05/2...,"1. La Corte d'Appello di Ancona, con sentenza ..."
23,Cass. penale,6366,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,cass pen 6366.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 06/12/2...,"1. La Corte d'Appello di Lecce, sezione distac..."
32,Cass. penale,20956,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,20956 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 03/05/2...,"1. La Corte di Appello di Palermo, pronunciand..."
19,Cass. penale,474,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,cass penale 474.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 09/11/2...,"1. Con sentenza del 3 aprile 2017, la Corte d'..."


10
70


#### Load Sumarries

Load summaries:

In [9]:
dfs = [
    'trial_0_score_0.91.csv',
    'trial_1_score_0.91.csv',
    'trial_2_score_0.91.csv',
    'trial_3_score_0.92.csv',
    'trial_4_score_0.93.csv',
    'trial_5_score_0.93.csv',
    'trial_6_score_0.91.csv',
    'trial_7_score_0.92.csv',
    'trial_8_score_0.93.csv',
]

base_dir = "/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/summaries/"
summary_df = pd.concat([pd.read_csv(base_dir + f) for f in dfs])

Get the best summary per sentenza:

In [10]:
idx = summary_df.groupby(['corte', 'numero sentenza'])['score'].transform(max) == summary_df['score']
summary_df = summary_df[idx]
print(len(summary_df))

81


Add the best summaries to the datasets:

In [11]:
tmp = summary_df[['numero sentenza', 'score', 'summary']]
train_val_df = train_val_df.merge(tmp, on=['numero sentenza'])
test_df = test_df.merge(tmp, on=['numero sentenza'])
print(len(train_val_df))
print(len(test_df))

72
11


#### Balance and Augment Datasets

Let's define a function to augment a dataframe with random chunks coming from the original text (facts) and the summary:

In [12]:
import random

random.seed(42)

def chunk_and_augment_df(df, chunk_size=512, num_rand_chunks=16, summary_score_threshold=0.85):
    # num_rand_chunks = 16
    # chunk_size = 512
    # score_threshold = 0.85
    train_val_entries = []
    for i, row in df.iterrows():
        facts = get_facts(clean_text(row['text']))
        summary = clean_text(row['summary'])
        for j in range(num_rand_chunks):
            # Add random chunks of the text to the set
            text_chunk = get_random_chunk(facts, chunk_size=chunk_size)
            row['chunk'] = text_chunk
            train_val_entries.append(row.copy())
            # Add random chunks of the summary, if the score is high enough
            if row['score'] > summary_score_threshold:
                summary_chunk = get_random_chunk(summary, chunk_size=chunk_size)
                row['chunk'] = summary_chunk
                train_val_entries.append(row.copy())
    return pd.DataFrame(train_val_entries)

for i, chunk in enumerate(chunk_and_augment_df(train_val_df)['chunk'].tolist()):
    print(chunk)
    print('-' * 80)
    if i > 20:
        break

 13207 del 27/01/2022, Premoli, Rv. 282936).  3.1.2 Esaminando, dunque, il caso concreto, la Corte territoriale - in riforma sul punto della sentenza di primo grado - ha riconosciuto le circostanze attenuanti generiche in considerazione dell'accertato concorso di colpa della persona offesa, ritenute prevalenti sulla circostanza aggravante dettata dal testo previgente dell'art. 589 c.p., comma 2, determinando la pena base in misura pari ad anni uno di reclusione, ridotta ad 8 mesi per il giudizio di prevalen
--------------------------------------------------------------------------------
 riferito delle circostanze attenuanti generiche in mesi otto di reclusione, nel confermare la condanna solidale dell'imputato e del responsabile civile al risarcimento del danno nei confronti delle costituite parti civili, da liquidarsi in separato giudizio, a carico del solo imputato - L'avvocato ha presentato ricorso per cassazione D.M.L., tramite il proprio difensore, articolando due motivi di impug

In [13]:
def balance_train_df(train_df, label_columns=label_columns):
    """ Balance the train_df by oversampling the minority class for each label in label_columns """
    balanced_dfs = []

    for label in label_columns:
        # Get the number of samples with label set to 1 and 0
        label_1_count = train_df[train_df[label] == 1].shape[0]
        label_0_count = train_df[train_df[label] == 0].shape[0]

        # Skip oversampling if all labels are the same
        if label_1_count == 0 or label_0_count == 0:
            balanced_dfs.append(train_df)
            continue

        # Determine the minority and majority class
        minority_class = 1 if label_1_count < label_0_count else 0
        majority_class = 1 if minority_class == 0 else 0

        # Calculate the oversampling factor
        oversampling_factor = label_0_count // label_1_count if minority_class == 1 else label_1_count // label_0_count

        # Oversample the minority class
        minority_df = train_df[train_df[label] == minority_class]
        oversampled_minority_df = pd.concat([minority_df] * oversampling_factor)

        # Combine the oversampled minority class with the majority class
        balanced_df = pd.concat([train_df[train_df[label] == majority_class], oversampled_minority_df])

        # # Print the number of samples with label set to 1 and 0
        # print(f"{label}: {len(balanced_df)} (0/1: {balanced_df[balanced_df[label] == 0].shape[0]} / {balanced_df[balanced_df[label] == 1].shape[0]})")

        balanced_dfs.append(balanced_df)

    # Merge the balanced dataframes for each label
    balanced_train_df = pd.concat(balanced_dfs)

    return balanced_train_df

tmp = chunk_and_augment_df(balance_train_df(train_val_df), num_rand_chunks=1)
for i, chunk in enumerate(tmp['chunk'].tolist()):
    print(chunk)
    print('-' * 80)
    if i > 10:
        break
print(len(train_val_df))
print(len(tmp))

tra disciplina, cioè in relazione al fatto contestato, prescindendo dai profili di maggiore o minor favore che astrattamente una disciplina presenti rispetto all'altra; tale analisi impone, in ossequio al rispetto del principio di legalità, di raffrontare peraltro le due discipline nella loro interezza, essendo vietato estrapolare dall'unae dall'altra le disposizioni più favorevoli in modo da delineare una terza disciplina (Sez. 4, n. 7961 del 17/01/2013, Capece, Rv. 255103; Sez. 4, n. 13207 del 27/01/2022,
--------------------------------------------------------------------------------
nto di correzione di errore materiale instaurato di fronte alla Corte territoriale, il giudice adito aveva omesso di disporre La sentenza impugnata limita limitata alla fattispecie di errore materiale denunciata dalle parti civili è l'unica via d'uscita per evitare l'esclusione delle parti civili da parte dell'amministrazione della Corte Suprema José Ramón Bauzá, consigliere comunale del PP, difende la 

#### Build HF Dataset

The Hugging Face dataset needs to be organized in a hierarchical way, with `text` and `labels`. Here is a function to do that based on a given dataframe:

In [14]:
from datasets import Dataset, Features, ClassLabel, Value, Sequence
from typing import Literal

def build_dataset_from_pandas(df, hierarchical=False, text_column_name: Literal['text', 'summary', 'facts'] = 'text'):
    if hierarchical:
        # Each entry of df['labels'] shall be a list of the column names whose indexes are 1
        df['labels'] = df[label_columns].apply(lambda x: [k for k, v in x.to_dict().items() if v == 1], axis=1)
        # Convert DataFrame to dictionary
        data_dict = {
            'text': [clean_text(t) for t in df[text_column_name].tolist()],
            'labels': df['labels'].tolist(),
        }
        # Define the features with nested structure
        features = Features({
            'text': Value('string'),
            'labels': Sequence(ClassLabel(names=label_columns)),
        })
        # Create the dataset
        return Dataset.from_dict(data_dict, features=features)
    data_dict = {k: df[k].tolist() for k in label_columns if k in df.columns}
    data_dict['text'] = [clean_text(t) for t in df[text_column_name].tolist()]
    return Dataset.from_dict(data_dict)

build_dataset_from_pandas(test_df)

Dataset({
    features: ['andamento retromarcia imputato', 'attraversamento corretto della vittima', 'attraversamento vittima con semaforo rosso', 'attraversamento vittima senza strisce o sbucata improvvisamente', 'centro abitato', 'colpo di sonno', 'eccesso verso imputato', 'eccesso verso vittima', 'fuga imputato', 'giorno', 'giovane eta imputato', 'guida brusca o manovra brusca della vittima', 'guida brusca o manovra brusca imputato', 'guida senza patente valida imputato', 'guida senza patente valida vittima', 'imputato perde controllo', 'imputato ubriaco o drogato', 'invasione corsia opposta imputato', 'lavori stradali di manutenzione', 'localita commerciale o luogo di lavoro', 'maltempo', 'mancanza cintura di sicurezza o casco vittima', 'neopat imputato', 'notte', 'ostacolo evitabile dall imputato', 'ostacolo prevedibile dall imputato', 'ostacolo prevedibile dalla vittima', 'piu imputati', 'poca illuminazione o scarsa visibilita', 'presenza rimorchio imputato', 'prossimita di un in

Following [this guide](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb) (and [this one](https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb) sometimes...).

In [15]:
labels = label_columns # [label for label in test_dataset.features.keys() if label not in ['text']]
num_labels = len(labels)
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
labels

['andamento retromarcia imputato',
 'attraversamento corretto della vittima',
 'attraversamento vittima con semaforo rosso',
 'attraversamento vittima senza strisce o sbucata improvvisamente',
 'centro abitato',
 'colpo di sonno',
 'eccesso verso imputato',
 'eccesso verso vittima',
 'fuga imputato',
 'giorno',
 'giovane eta imputato',
 'guida brusca o manovra brusca della vittima',
 'guida brusca o manovra brusca imputato',
 'guida senza patente valida imputato',
 'guida senza patente valida vittima',
 'imputato perde controllo',
 'imputato ubriaco o drogato',
 'invasione corsia opposta imputato',
 'lavori stradali di manutenzione',
 'localita commerciale o luogo di lavoro',
 'maltempo',
 'mancanza cintura di sicurezza o casco vittima',
 'neopat imputato',
 'notte',
 'ostacolo evitabile dall imputato',
 'ostacolo prevedibile dall imputato',
 'ostacolo prevedibile dalla vittima',
 'piu imputati',
 'poca illuminazione o scarsa visibilita',
 'presenza rimorchio imputato',
 'prossimita di

### Setup Model and Tokenizer (and PEFT)

Start from the tokenizer:

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# model_name = "iproskurina/tda-itabert-ita-cola"
# model_name = "ARTeLab/mbart-summarization-mlsum"
# model_name = "papluca/xlm-roberta-base-language-detection" # Best one so far
model_name = "classla/xlm-roberta-base-multilingual-text-genre-classifier"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    # max_length=512, # At least 512
)

tokenizer_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

### Tokenize Datasets and Generate K-folds

Let's setup a CV generator:

In [17]:
import torch
import numpy as np
from datasets import Features, Value, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold
import os

def preprocess_function(examples, text_column_name="text"):
    # Take a batch of texts
    text = [text for text in examples["text"]]
    # Encode them
    encoding = tokenizer(text, padding="max_length", truncation=True)
    # Add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # Create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # Fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    encoding["labels"] = labels_matrix.tolist()
    return encoding


def k_fold_tokenized_datasets(
        n_splits=5,
        chunk_size=tokenizer.max_model_input_sizes['xlm-roberta-base'],
        num_rand_chunks_train=1,
        num_rand_chunks_val=10,
):
    gkf = GroupKFold(n_splits=n_splits)
    k_splits = gkf.split(train_val_df, groups=train_val_df['numero sentenza'])

    # Tokenize test dataset and get the columns to remove
    tmp = chunk_and_augment_df(
        test_df,
        chunk_size=chunk_size,
        num_rand_chunks=10,
        summary_score_threshold=1.1, # NOTE: do NOT include any summary in val set (all scores are <= 1.0)
    )
    test_dataset = build_dataset_from_pandas(tmp, text_column_name='facts')
    cols_to_remove = [c for c in test_dataset.column_names if c not in labels] + labels
    test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=cols_to_remove)

    for train_idx, val_idx in k_splits:
        train_df = train_val_df.iloc[train_idx, :].copy()
        val_df = train_val_df.iloc[val_idx, :].copy()
        # Balance and augment training dataset
        train_df = chunk_and_augment_df(
            balance_train_df(train_df),
            chunk_size=chunk_size,
            num_rand_chunks=num_rand_chunks_train,
            summary_score_threshold=0.85,
        )
        # Chunk-ify val set
        val_df = chunk_and_augment_df(
            val_df,
            chunk_size=chunk_size,
            num_rand_chunks=num_rand_chunks_val,
            summary_score_threshold=1.1, # NOTE: do NOT include any summary in val set (all scores are <= 1.0)
        )
        # Generate the dataset from the Dataframes and the column "chunk"
        train_dataset = build_dataset_from_pandas(train_df, text_column_name="chunk")
        val_dataset = build_dataset_from_pandas(val_df, text_column_name="chunk")
        # Assemble and tokenize the datasets accordingly
        tokenized_dataset = DatasetDict({
            'train': train_dataset.map(preprocess_function, batched=True, remove_columns=cols_to_remove),
            'val': val_dataset.map(preprocess_function, batched=True, remove_columns=cols_to_remove),
            'test': test_dataset,
        })
        tokenized_dataset.set_format('torch')
        yield tokenized_dataset

base_dir = "/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/datasets/"
for k, tokenized_dataset in enumerate(k_fold_tokenized_datasets(n_splits=5)):
    print('-' * 80)
    print(f'k = {k}')
    print('-' * 80)
    print(tokenized_dataset)
    ds_name = f"{base_dir}/dataset-omicidio-stradale-fold-{k}"
    if not os.path.exists(ds_name):
        tokenized_dataset.save_to_disk(ds_name)

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Map:   0%|          | 0/8100 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

--------------------------------------------------------------------------------
k = 0
--------------------------------------------------------------------------------
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8100
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/8100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

--------------------------------------------------------------------------------
k = 4
--------------------------------------------------------------------------------
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8369
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/8369 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/140 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/110 [00:00<?, ? examples/s]

In [18]:
example = tokenized_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [19]:
tokenized_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [20]:
tokenized_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8369
})

### Get Model

In [21]:
def find_target_modules(model):
    """ source: https://www.reddit.com/r/LocalLLaMA/comments/15sgg4m/what_modules_should_i_target_when_training_using/ """

    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear'
        if "Linear" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)

There is a bug in the model initialization when more labels then the pretrained ones are used (there is no initialization of some parameters, leading to NaN weights). The work-around is to manually run initialization:

In [21]:
# import torch.nn as nn

# classifier = model.classifier.out_proj

# # Xavier initialization for the weights
# nn.init.xavier_uniform_(classifier.weight)

# # Also initialize the biases to zero if required
# nn.init.zeros_(classifier.bias)

# torch.isna(classifier.weight).any()

In [22]:
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType

device = "cuda" if torch.cuda.is_available() else "cpu"

def get_model(model_name, freeze_encoder=True, is_peft=False):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        problem_type="multi_label_classification",
        num_labels=len(labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True,
        # load_in_8bit=True if torch.cuda.is_available() else False,
    )
    print("model.config.max_length:", model.config.max_length)
    print("model.config.max_position_embeddings:", model.config.max_position_embeddings)
    # TODO: Because of the bug, we need to manually inizialize the classifier.
    # However, the modules names can vary, so the following shall be made
    # generic, somehow
    classifier = model.classifier.out_proj
    # Xavier initialization for the weights
    nn.init.xavier_uniform_(classifier.weight)
    # Also initialize the biases to zero
    nn.init.zeros_(classifier.bias)
    print("Any NaN in the classifier weights: ", torch.isnan(classifier.weight).any())
    # Freeze all non-classifier layers
    if freeze_encoder:
        for name, param in model.named_parameters():
            if "classifier" not in name:
                param.requires_grad = False
    if is_peft:
        peft_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=8, # Original: 8
            lora_alpha=64, # Original: 64
            lora_dropout=0.1,
            target_modules=find_target_modules(model),
            inference_mode=False,
        )
        model = get_peft_model(model, peft_config)
        model.print_trainable_parameters()
    return model

In [23]:
# from peft import get_peft_model, LoraConfig, TaskType

# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,
#     r=8, # Original: 8
#     lora_alpha=64, # Original: 64
#     lora_dropout=0.1,
#     target_modules=find_target_modules(model),
#     inference_mode=False,
# )
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [23]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import evaluate

accuracy = evaluate.load("accuracy")

def multi_label_metrics(predictions, labels, threshold=0.5):
    """ source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/ """
    # First, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # Next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # Finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    # Return as dictionary
    metrics = {
        'f1': f1_micro_average,
        'roc_auc': roc_auc,
        'accuracy': accuracy,
    }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids,
    )
    return result

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Start Training

In [25]:
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
from transformers import TrainingArguments, Trainer
from datasets import load_from_disk

metric_name = "f1"
version = "v0.2"
test_scores_table = []

# k_folds = k_fold_tokenized_datasets(n_splits=5, num_rand_chunks_train=64)
# for k, tokenized_dataset in enumerate(k_folds):

base_dir = "/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/datasets/"
for k in range(5):
    ds_name = f"{base_dir}/dataset-omicidio-stradale-fold-{k}"
    tokenized_dataset = load_from_disk(ds_name)
    # Get model
    model = get_model(model_name)
    # Setup trainer
    model_id = f"RuleBert-{version}-k{k}"
    output_dir = f"{model_id}"
    hub_model_id = f"ribesstefano/{model_id}"
    training_args = TrainingArguments(
        output_dir=output_dir,
        # Optimizer settings
        learning_rate=2e-5,
        optim="adamw_torch",
        lr_scheduler_type="linear", # "cosine", # Default: "linear"
        weight_decay=0.015,
        # Batching and device-related settings
        per_device_train_batch_size=4,
        per_device_eval_batch_size=64,
        gradient_accumulation_steps=1,
        eval_accumulation_steps=4,
        fp16=True if torch.cuda.is_available() else False,
        # num_train_epochs=1,
        # save_strategy="epoch",
        # evaluation_strategy="epoch",
        save_total_limit=1,
        data_seed=42,
        evaluation_strategy="steps",
        save_strategy="steps",
        max_steps=500 * 16,
        eval_steps=250,
        save_steps=500,
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        # Hub information configs
        push_to_hub=True, # NOTE: Done manually further down
        hub_token=HF_TOKEN,
        hub_model_id=hub_model_id,
        hub_strategy="checkpoint", # NOTE: Allows to resume training from last checkpoint
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    # Train and push to hub
    trainer.train(
        # resume_from_checkpoint="last-checkpoint",
    )
    trainer.push_to_hub(
        commit_message="Initial version",
        model_name=hub_model_id,
        license="mit",
        finetuned_from=model_name,
        tasks=["Text Classification"],
        tags=["Italian", "legal ruling"],
        # dataset=ds_name,
        # dataset_args=ds_config,
    )
    tmp = []
    # Predict on test dataset
    test_scores = trainer.predict(tokenized_dataset["test"]).metrics
    test_scores['k'] = k
    test_scores['dataset'] = 'test'
    test_scores_table.append(test_scores)
    tmp.append(test_scores)
    # Predict on val dataset
    test_scores = trainer.predict(tokenized_dataset["val"]).metrics
    test_scores['k'] = k
    test_scores['dataset'] = 'val'
    test_scores_table.append(test_scores)
    tmp.append(test_scores)
    # Predict on train dataset
    test_scores = trainer.predict(tokenized_dataset["train"]).metrics
    test_scores['k'] = k
    test_scores['dataset'] = 'train'
    test_scores_table.append(test_scores)
    tmp.append(test_scores)

    print('-' * 80)
    print(f"K = {k}")
    display(pd.DataFrame(tmp).round(4))
    print('-' * 80)
    # Delete model and continue with the next fold in CV
    del model
pd.DataFrame([s for s in test_scores_table]).to_csv(f"/content/drive/MyDrive/Colab Notebooks/ml_for_justice/scores-{version}.csv", index=False)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at classla/xlm-roberta-base-multilingual-text-genre-classifier and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([44]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([44, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.config.max_length: 20
model.config.max_position_embeddings: 514
Any NaN in the classifier weights:  tensor(False)


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
250,0.388200,0.369683,0.507496,0.676413,0.006667
500,0.340600,0.362992,0.513529,0.676784,0.020000
750,0.338200,0.367448,0.497783,0.672086,0.000000
1000,0.336500,0.371066,0.502530,0.673393,0.006667


In [ ]:
pd.DataFrame([s for s in test_scores_table]).to_csv("/content/drive/MyDrive/Colab Notebooks/ml_for_justice/scores.csv", index=False)

for s in test_scores_table:
    print(s)

In [ ]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_f1': 0.23813488759367193,
 'eval_roc_auc': 0.5676009342676009,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.1331,
 'eval_samples_per_second': 946.807,
 'eval_steps_per_second': 15.029,
 'epoch': 2.0}

## Inference

For the evaluation, reproduce [this detailed analysis](https://colab.research.google.com/github/samlowe/go_emotions-dataset/blob/main/eval-roberta-base-go_emotions.ipynb#scrollTo=571CYWiY6pS7).

In [44]:
def get_predicted_labels(text, threshold=0.5):
    encoding = tokenizer(text, return_tensors="pt", padding="max_length")
    encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}
    outputs = trainer.model(**encoding)
    logits = outputs.logits
    # Apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.cpu()) # .squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= threshold)] = 1
    # Turn predicted ID's into actual label names
    labeled_predictions = []
    for pred in predictions:
        tmp = [id2label[idx] for idx, label in enumerate(pred) if label == 1.0]
        labeled_predictions.append(tmp)
    return labeled_predictions

examples = [
    "L'imputato ha contestato una colpa generica e la violazione dell'art. 141 C.d. S. perché alla guida della sua autovettura, nel percorrere la strada provinciale, aveva tenuto una velocità non adeguata.",
    "L'imputato, sotto effetto di alcool, ha contestato una colpa generica e la violazione dell'art. 141 C.d. S. perché alla guida della sua autovettura, nel percorrere la strada provinciale fuori cittá, aveva tenuto una velocità non adeguata di giorno.",
]
predicted_labels = get_predicted_labels(examples, threshold=0.4)

for pred in predicted_labels:
    for l in pred:
        print('*', l)
    print('-' * 80)

* notte
* ostacolo evitabile dall imputato
* ostacolo prevedibile dall imputato
* violazione stop o semaforo o codice della strada da parte imputato
--------------------------------------------------------------------------------
* notte
* ostacolo evitabile dall imputato
* ostacolo prevedibile dall imputato
* violazione stop o semaforo o codice della strada da parte imputato
--------------------------------------------------------------------------------


## Summarizing Texts

In [ ]:
from transformers import pipeline

pipe = pipeline("summarization", model="ARTeLab/mbart-summarization-mlsum", device="cuda")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

In [ ]:
for sample in dataset:
    text = sample['text']
    sentences = text.split(sentence_div)
    clean_sentences = [s.split(section_name_div)[-1].strip() for s in sentences]
    summaries = pipe(clean_sentences[:2])
    for c, s in zip(clean_sentences[:2], summaries):
        print('-' * 80)
        print(s['summary_text'])
        print(c)
    break

Your max_length is set to 1024, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 1024, but your input_length is only 169. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=84)


--------------------------------------------------------------------------------
La Corte d'Appello di Ancona, con sentenza nel 2022, confermava la sentenza del GUP davanti al Tribunale Urbino di condanna di h.m. quale responsabile del reato di omicidio colposo ai danni di F.C., deceduto in un sinistro stradale, anche relativamente alle statuizioni civili, lo condannava al pagamento delle spese sostenute dalle parti civili per il giudizio di appello.
1. La Corte d'Appello di Ancona, con sentenza in data 12 aprile 2022, confermava la sentenza del GUP presso il Tribunale Urbino di condanna di h.m. quale responsabile del reato di omicidio colposo ai danni di F.C., deceduto in un sinistro stradale, anche relativamente alle statuizioni civili e lo condannava al pagamento delle spese sostenute dalle parti civili per il giudizio di appello.
--------------------------------------------------------------------------------
L'imputato ha contestato una colpa generica e la violazione dell'art. 141

In [ ]:
for sample in dataset:
    text = sample['text']
    sentences = text.split(section_div)
    clean_sentences = [s.split(section_name_div)[-1].strip().replace(sentence_div, ' ') for s in sentences]
    clean_sentences = [s for s in clean_sentences if s.strip() != '']
    for s in clean_sentences[3:4]:
        print('*', s)
    summaries = pipe(clean_sentences[3:4])
    for c, s in zip(clean_sentences[3:4], summaries):
        print('=' * 80)
        print(s['summary_text'])
        print('-' * 80)
        print(c)
    break

* 1. Il ricorso non è fondato.   2. Con il primo motivo si contesta l'affermazione di responsabilità, alla quale la Corte territoriale era pervenuta senza valutare la ricostruzione alternativa operata dall'imputato in ordine alla dinamica del sinistro.   Il motivo è privo di pregio.   Va innanzi tutto ricordato che si verte in ipotesi di "doppia conforme", ove l'obbligo motivazionale gravante sul giudice di secondo grado è meno stringente, dovendosi fare riferimento ad un complesso motivazionale coerente ed organico, rappresentato da entrambe le sentenze.   E va altresì rimarcato che il principio secondo cui la condanna può essere pronunciata solo se l'imputato risulti colpevole "oltre ogni ragionevole dubbio", non può essere utilizzato, nel giudizio di legittimità, per valorizzare e rendere decisiva una ricostruzione alternativa del fatto emersa in sede di merito su segnalazione della difesa, se tale differente prospettazione sia stata oggetto di puntuale e motivata disamina da parte 

RuntimeError: ignored